# SmartFigure context-managed parameters demo

This notebook demonstrates the module-level `params[...]` proxy and `parameter(...)` helper, with an emphasis on how they respect the `with fig:` context stack.

## Imports

We use the import block requested in the blueprint instructions.

In [ ]:
from pathlib import Path
import sys
ROOT = Path.cwd().resolve().parents[1]
sys.path.insert(0, str(ROOT))
from gu_toolkit.SmartFigure import  *
from gu_toolkit.ParamEvent import  *
from gu_toolkit.SmartSlider import  *

import sympy as sp


## Basic figure with explicit parameters

We create a figure, plot a parametric expression, and then access the parameter via the module-level `params` proxy inside the `with fig:` context.

In [ ]:
x, a = sp.symbols('x a')
fig = SmartFigure(x_range=(-6, 6), y_range=(-3, 3))
with fig:
    fig.plot(x, a * sp.sin(x), parameters=[a], id='a_sin')
    params[a].value = 2
fig

## Programmatic updates to parameter values and limits

The `params` proxy returns a `ParamRef`, so you can mutate its `value`, `min`, `max`, and `step` directly. The assignment sugar `params[a] = 5` updates the `value`.

In [ ]:
with fig:
    params[a].min = -5
    params[a].max = 5
    params[a].step = 0.5
    params[a] = 3
    params[a].value

## Strict lookup and explicit creation

`params[...]` is a strict lookup: it does **not** create missing parameters. Use `parameter(...)` to create parameters explicitly.

In [ ]:
b = sp.symbols('b')
with fig:
    try:
        params[b]  # strict lookup: should raise KeyError
    except KeyError as exc:
        print('Strict lookup:', exc)

    parameter(b, min=-2, max=2, step=0.25, value=1)
    print('Created b ->', params[b].value)

## Context-aware callbacks

`SmartFigure.add_param_change_hook` wraps your callback in the local `with fig:` context. That means module-level `params[...]` inside the callback resolves to the figure that triggered the event.

In [ ]:
c = sp.symbols('c')
fig2 = SmartFigure(x_range=(-4, 4), y_range=(-2, 2))
with fig2:
    fig2.plot(x, c * sp.cos(x), parameters=[c], id='c_cos')
    params[c].value = 1

log = []

def hook_fig1(event):
    log.append(("fig1", params[a].value, a in params, b in params))

def hook_fig2(event):
    log.append(("fig2", params[c].value, c in params))

fig.add_param_change_hook(hook_fig1, run_now=False)
fig2.add_param_change_hook(hook_fig2, run_now=False)

with fig:
    params[a] = 4
with fig2:
    params[c] = 2

log

## Takeaways

- `params[...]` respects the active `with fig:` context and never auto-creates parameters.
- `parameter(...)` is the explicit creation helper.
- Callbacks triggered by a figure run inside that figure's context, so module-level parameter access is safe and unambiguous.